# Simplex tableau

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

from dataclasses import dataclass

import numpy as np
from numpy.linalg import LinAlgError




In this lab, you will learn how to **read and use a simplex tableau**.
You will identify the **basic variables** and their rows, rebuild the tableau from its
definition (B^{-1}A, B^{-1}b, reduced costs, negative objective), and recover the **basic feasible
solution** and its objective value. You will compute **reduced costs** for nonbasic variables,
derive the corresponding **basic directions**, and verify that reduced costs match the
**directional derivatives** along those directions. The goal is to connect the algebra of the
tableau to the geometry of linear optimization problems so that each entry in the tableau has a clear,
practical meaning for optimality tests and pivot choices.

Consider the optimization problem in standard form characterized by the
following data:
$$
A=
\begin{pmatrix*}
1&-1&1&0\\
1&1&0&1
\end{pmatrix*}
;
b=
\begin{pmatrix*}
2\\
6
\end{pmatrix*}
;
c=
\begin{pmatrix*}
-2\\
-1\\
0\\
0
\end{pmatrix*}.
$$
Consider as well the following tableau:
$$
\begin{array}{|c c c c | c |}
\hline
x_0& x_1& x_2& x_3 & \\
\hline
1&-1&1&0  &2\\
0&2&-1&1  &4 \\
\hline
0&-3&2&0  &4 \\
\hline
\end{array}
$$


1. What are the basic variables associated with this tableau?
1. What are the variables corresponding to the each row of the
tableau?
1. What is the basic matrix $B$?
1. Verify that the tableau is valid, in the sense that it corresponds to its definition.
1. What is the vertex $x$ corresponding to this tableau?
1. What is the value of the objective function at that vertex?
1. What are the reduced costs of the non basic variables?
1. What are the basic directions?
1. Verify that the values of the reduced costs in the tableau are
consistent with the basic directions.

Prepare the data

In [ ]:
standard_a = ...
standard_b = ...
standard_c = ...


In [ ]:
the_tableau = ...


print(the_tableau)



We define a data structure that stores the row and column of an element of the matrix

In [ ]:
@dataclass
class RowColumn:
    row: int
    column: int



We create an element as follows:

In [ ]:
element = RowColumn(row=3, column=5)
print(element)


We can access the attributes as follows:

In [ ]:
print(f'Row: {element.row}')
print(f'Column: {element.column}')



We start by writing a function that identifies the column associated with the basic variables. They are the columns
of the identity matrix. And the location of the only non zero element (1) identifies the corresponding row.

Fill in the ...

In [ ]:
def identify_basic_variables(tableau: np.ndarray) -> list[RowColumn]:
    """Function that identifies the column associated with the basic variables.

    :param tableau: simplex tableau
    :return: a list reporting the position of the 1's in the tableau corresponding to the basic variable.
    """
    n_rows, n_columns = tableau.shape
    # We need to identify where the columns of the identify matrix are, and where the ones in those columns are
    # located.
    ones = []
    for col_index in range(n_columns):
        column = tableau[:, col_index]
        # Check if there's exactly one entry with 1 and the rest are 0
        if ...:


            # Find the index of the 1
            row_index = ...


            ones.append(RowColumn(row_index, col_index))
    return ones



Test the function.

In [ ]:
the_basic_variables = identify_basic_variables(the_tableau)
the_basic_indices = [element.column for element in the_basic_variables]
for element in the_basic_variables:
    print(
        f'Variable {element.column} is in the basis and corresponds to row {element.row}'
    )


The basic matrix is therefore:

In [ ]:
the_basic_matrix = standard_a[:, the_basic_indices]
print(f'Basic matrix:\n{the_basic_matrix}')



We write a function that builds the tableau using its definition:

- Upper-left part: $B^{-1}A$.
- Upper-right part: $B^{-1}b$.
- Lower-left: $c^T - c_B^TB^{-1}A$.
- Lower-right: $-c_B^TB^{-1}b$.

Fill in the ...

In [ ]:
def build_tableau(
    matrix: np.ndarray,
    right_hand_side: np.ndarray,
    objective: np.ndarray,
    basic_indices: list[int],
) -> np.ndarray | None:
    """Function that builds the tableau using its definition

    :param matrix: constraint matrix (standard form)
    :param right_hand_side: right hand side
    :param objective: coefficients of the objective function
    :return: simplex tableau
    """
    n_constraints, n_variables = matrix.shape

    if len(right_hand_side) != n_constraints:
        error_msg = (
            f'Inconsistent dimensions {len(right_hand_side)} and {n_constraints}'
        )
        raise ValueError(error_msg)

    if len(objective) != n_variables:
        error_msg = f'Inconsistent dimensions {len(objective)} and {n_variables}'
        raise ValueError(error_msg)

    if len(basic_indices) != n_constraints:
        error_msg = f'Inconsistent dimensions {len(basic_indices)} and {n_constraints}'
        raise ValueError(error_msg)

    wrong_indices = [
        index for index in basic_indices if index < 0 or index >= n_variables
    ]
    if wrong_indices:
        error_msg = f'Wrong basic indices: {wrong_indices}'
        raise ValueError(error_msg)

    basic_matrix = ...
    basic_costs = ...

    upper_left = ...




    upper_right = ...


    lower_left = ...
    lower_right = ...
    tableau = np.empty((n_constraints + 1, n_variables + 1))
    tableau[:n_constraints, :n_variables] = upper_left
    tableau[:n_constraints, n_variables] = upper_right
    tableau[n_constraints, :n_variables] = lower_left
    tableau[n_constraints, n_variables] = lower_right
    return tableau



We test the function.

In [ ]:
built_tableau = build_tableau(
    matrix=standard_a,
    right_hand_side=standard_b,
    objective=standard_c,
    basic_indices=the_basic_indices,
)
print('The tableau built from its definition is:')
print(built_tableau)
print('It must correspond to the tableau provided as input.')
print(the_tableau)



We now write a function that constructs the feasible basic solution corresponding to the tableau.
Fill in the ...

In [ ]:
def feasible_basic_solution(tableau: np.ndarray) -> np.ndarray:
    """Function that constructs the feasible basic solution corresponding to the tableau.

    :param tableau: simplex tableau
    :return: feasible basic solution
    """
    n_rows, n_columns = tableau.shape
    n_variables = n_columns - 1
    basic_variables = identify_basic_variables(tableau)
    result = ...


    return result



Test the function

In [ ]:
the_feasible_basic_solution = feasible_basic_solution(tableau=the_tableau)
print(f'Solution: {the_feasible_basic_solution}')



Write a function that returns the value of the objective function at the vertex.
Fill in the ....

In [ ]:
def value_objective_function(tableau: np.ndarray) -> float:
    """
    It is simply the opposite of the lower right cell of the tableau.
    """
    return ...



Test the function.

In [ ]:
the_objective_value = value_objective_function(tableau=the_tableau)
print(f'Value of the objective function: {the_objective_value}')



Write a function that calculates the reduced costs of the non basic variables.
Fill in the ....

In [ ]:
def reduced_costs(tableau: np.ndarray) -> dict[int, float]:
    """
    Function that calculates the reduced costs of the non basic variables.
    :param tableau: the simplex tableau
    :return: a dict mapping the indices of the non basic variables and the reduced costs.
    """
    n_rows, n_columns = tableau.shape
    n_variables = n_columns - 1
    basic_variables = identify_basic_variables(tableau)
    non_basic_variables = list(
        set(range(n_variables)) - set([element.column for element in basic_variables])
    )
    result = ...


    return result



Test the function

In [ ]:
the_reduced_costs = reduced_costs(tableau=the_tableau)
print(f'Reduced costs: {the_reduced_costs}')



Write a function that builds the basic directions.
Fill in the ...

In [ ]:
def extract_basic_directions(tableau: np.ndarray) -> dict[int, np.ndarray]:
    """
    Each column of non basic variable corresponds to the opposite of the basic part of the basic directions.
    :param tableau: simplex tableau.
    :return: dict mapping the non basic variables with the basic directions.
    """
    n_rows, n_columns = tableau.shape
    n_variables = n_columns - 1
    basic_variables = identify_basic_variables(tableau)
    non_basic_variables = list(
        set(range(n_variables)) - set([element.column for element in basic_variables])
    )
    result = ...







    return result



Test the function

In [ ]:
the_basic_directions = extract_basic_directions(tableau=the_tableau)
print(f'Basic directions: {the_basic_directions}')


Finally, we need to verify that the values of the reduced costs in the tableau are
consistent with the basic directions. It means that the reduced costs are the directional derivatives of the
function in the direction of the basic directions.
Fill in the ....

In [ ]:

for index, basic_direction in the_basic_directions.items():
    reduced_cost = the_reduced_costs[index]
    directional_derivative = ...


    print(
        f'Non basic variable {index}: reduced cost: {reduced_cost}, directional derivative: {directional_derivative}'
    )